In [ ]:
%%bash
# preamble script to check and install AMUSE components if necessary

# required packages for this tutorial:
PACKAGES="amuse-framework"
# skip in case a full development install is present
pip show amuse-devel && exit 0
for package in ${PACKAGES} 
do
  pip show ${package} || pip install ${package}
done

In [ ]:
# the following fixes are highly recommended

#allow oversubscription for openMPI
import os
os.environ["OMPI_MCA_rmaps_base_oversubscribe"]="true"

# use lower cpu resources for idle codes
from amuse.support import options
options.GlobalOptions.instance().override_value_for_option("polling_interval_in_milliseconds", 10)


In [ ]:
%matplotlib inline
from matplotlib import pyplot
import numpy

All calculations in AMUSE are done with quantities having units. These quantities and their units are implemented as python classes and can be used almost everywere you would normaly use a number (or a numpy array). 

As we will do in all tutorials we start by importing everything from amuse.lab:

In [ ]:
from amuse.lab import *

You can create a quantity by typing a number and combining it with a unit using the bar `|` operator.

In [ ]:
1.989e30 | units.kg

Most operations you can do on numbers, you can also perform on quantities. For example, assuming the earth is a sphere, let's calculate some properties:

In [ ]:
earth_radius = 6384 | units.km
print("diameter :", earth_radius * 2)
print("surface area :", 4.0 * numpy.pi * earth_radius**2)
print("volume :", 4.0 / 3.0 * numpy.pi * earth_radius**3)

It is also possible to combine quantities with different units in a calculation. To continue our properties of the earth example, lets calcute it's density:

In [ ]:
earth_mass = 5.97219e24 | units.kg
earth_volume = 4.0 * numpy.pi * earth_radius**3 / 3
density = earth_mass / earth_volume
print("density :", earth_mass / earth_volume)

Note that density has a *numerical value* and a *unit*

In [ ]:
print("numerical value:", density.number)
print("unit:", density.unit)
print("simplified:", density.in_base())

You will want to interact with other python libraries that simply cannot handle units. For those cases you can convert the quantity into a value of a specific unit:

In [ ]:
print("earth mass in gram :", earth_mass.value_in(units.g))

Astrophysical units are also supported:

In [ ]:
print("earth mass in solar masses :", earth_mass.value_in(units.MSun))

To also print the unit, you can use a conversion function:

In [ ]:
earth_mass2 = earth_mass.in_(units.MSun)
print("earth mass :", earth_mass2)

Note that while the two quantities have different representation, they are still the same (though this is true within numerical precision):

In [ ]:
print(earth_mass == earth_mass2)

Numpy arrays and python lists can also be converted to vector quantities. Once converted, the resulting quantities support a lot of numpy operations

In [ ]:
masses = [641.85, 4868.5, 5973.6, 102430, 86832, 568460, 1898600] | (1e21 * units.kg)
radii = [0.532, 0.950, 1, 3.86, 3.98, 9.14, 10.97] | (6384 * units.km)
print("sum of planet masses: ", masses.sum().in_(units.MSun))
print("planet diameters: ", (radii * 2))

You can create your own unit with a new name using the `units.named` function. This functions takes the name of the unit, a symbol for printing and the unit it is based on. You can define a unit to represent the volume of the earth

In [ ]:
earth_volume_unit = units.named('Vol-Earth', 'EarthVol', earth_volume.to_unit())
print(earth_volume.in_(earth_volume_unit))

Do you note something odd? You have to be careful with numerical precision!

Most operations on a vector quantity are elementwise. We can do some operations on the array of planet masses and raddii we defined earlier. (As AMUSE uses numpy internally for these operations we refer to the numpy documentation, if you want to learn more)

In [ ]:
volumes = 4.0 / 3.0 * numpy.pi * radii**3
earth_density = earth_mass / earth_volume
print("volumes :", volumes.in_(earth_volume_unit))
print("densities :", (masses / volumes))

Quantities become normal numbers or numpy arrays when the units cancel out in a calcution (You can use this fact, to replace a `value_in` function call with a division):

In [ ]:
print(volumes / earth_volume)

Operations with incompatible units will fail:

In [ ]:
print(earth_mass + earth_volume)